**Compute Key-Performance Indicators (KPIs) from the MLflow server.**

# IMPORTS

In [1]:
import mlflow
import pandas as pd
import plotly.express as px

# OPTIONS

In [2]:
pd.options.display.max_columns = None

# CONFIGS

In [3]:
MAX_RESULTS = 100
TRACKING_URI = "http://localhost:5000"
REGISTRY_URI = "http://localhost:5000"

# CLIENTS

In [4]:
client = mlflow.tracking.MlflowClient(tracking_uri=TRACKING_URI, registry_uri=REGISTRY_URI)

# INDICATORS

In [5]:
experiments = client.search_experiments(
    view_type=mlflow.entities.ViewType.ALL, max_results=MAX_RESULTS, order_by=["creation_time DESC"]
)
experiments = [dict(experiment) for experiment in experiments]
experiments = pd.DataFrame(experiments).assign(
    creation_time=lambda data: pd.to_datetime(data["creation_time"], unit="ms"),
    last_update_time=lambda data: pd.to_datetime(data["last_update_time"], unit="ms"),
)
print(experiments.shape)
experiments.head()

(1, 7)


,artifact_location,creation_time,experiment_id,last_update_time,lifecycle_stage,name,tags
0,file:///home/fmind/mlops-python-package/mlruns...,2024-12-14 14:35:02.439,300971336194126583,2024-12-14 14:35:02.439,active,bikes,{}


In [6]:
runs = client.search_runs(
    experiment_ids=experiments["experiment_id"].unique(),
    run_view_type=mlflow.entities.ViewType.ALL,
    max_results=MAX_RESULTS,
    order_by=["created DESC"],
)
runs = [dict(run.info) | dict(run.data) for run in runs]
runs = pd.DataFrame(runs).assign(
    end_time=lambda data: pd.to_datetime(data["end_time"], unit="ms"),
    start_time=lambda data: pd.to_datetime(data["start_time"], unit="ms"),
    run_time_secs=lambda data: (data["end_time"] - data["start_time"]).map(
        lambda t: t.total_seconds()
    ),
)
runs = pd.concat([runs, pd.json_normalize(runs["tags"])], axis="columns")
print(runs.shape)
runs.head()

(27, 29)


,artifact_uri,end_time,experiment_id,lifecycle_stage,run_id,run_name,run_uuid,start_time,status,user_id,metrics,params,tags,run_time_secs,mlflow.user,mlflow.source.name,mlflow.source.type,mlflow.project.entryPoint,mlflow.source.git.commit,mlflow.source.git.repoURL,mlflow.gitRepoURL,mlflow.runName,mlflow.project.env,mlflow.project.backend,estimator_name,estimator_class,mlflow.log-model.history,mlflow.autologging,mlflow.parentRunId
0,file:///home/fmind/mlops-python-package/mlruns...,2024-12-14 14:42:28.056,300971336194126583,active,9923cb99c1824857b68c0bedb63f8446,Explanations,9923cb99c1824857b68c0bedb63f8446,2024-12-14 14:42:16.077,FINISHED,fmind,{},{'conf_file': 'confs/explanations.yaml'},"{'mlflow.user': 'fmind', 'mlflow.source.name':...",11.979,fmind,file:///home/fmind/mlops-python-package,PROJECT,main,1a4b8a25a32f0e933a7db9ec47f763cc7bd17d1c,git@github.com:fmind/mlops-python-package,git@github.com:fmind/mlops-python-package,Explanations,virtualenv,local,NaN,NaN,NaN,NaN,NaN
1,file:///home/fmind/mlops-python-package/mlruns...,2024-12-14 14:42:13.733,300971336194126583,active,bfce074e34834054b84e5794450a4fa2,Evaluations,bfce074e34834054b84e5794450a4fa2,2024-12-14 14:42:07.488,FINISHED,fmind,"{'example_count': 13903.0, 'mean_absolute_erro...",{'conf_file': 'confs/evaluations.yaml'},"{'mlflow.user': 'fmind', 'mlflow.source.name':...",6.245,fmind,file:///home/fmind/mlops-python-package,PROJECT,main,1a4b8a25a32f0e933a7db9ec47f763cc7bd17d1c,git@github.com:fmind/mlops-python-package,git@github.com:fmind/mlops-python-package,Evaluations,virtualenv,local,NaN,NaN,NaN,NaN,NaN
2,file:///home/fmind/mlops-python-package/mlruns...,2024-12-14 14:42:05.031,300971336194126583,active,5ebe28d67ed54300a177eb61825a6688,Inference,5ebe28d67ed54300a177eb61825a6688,2024-12-14 14:41:59.607,FINISHED,fmind,{},{'conf_file': 'confs/inference.yaml'},"{'mlflow.user': 'fmind', 'mlflow.source.name':...",5.424,fmind,file:///home/fmind/mlops-python-package,PROJECT,main,1a4b8a25a32f0e933a7db9ec47f763cc7bd17d1c,git@github.com:fmind/mlops-python-package,git@github.com:fmind/mlops-python-package,Inference,virtualenv,local,NaN,NaN,NaN,NaN,NaN
3,file:///home/fmind/mlops-python-package/mlruns...,2024-12-14 14:41:57.245,300971336194126583,active,f29b930df5a54f1fa9ec5fe4b27df8b9,Promotion,f29b930df5a54f1fa9ec5fe4b27df8b9,2024-12-14 14:41:52.639,FINISHED,fmind,{},{'conf_file': 'confs/promotion.yaml'},"{'mlflow.user': 'fmind', 'mlflow.source.name':...",4.606,fmind,file:///home/fmind/mlops-python-package,PROJECT,main,1a4b8a25a32f0e933a7db9ec47f763cc7bd17d1c,git@github.com:fmind/mlops-python-package,git@github.com:fmind/mlops-python-package,Promotion,virtualenv,local,NaN,NaN,NaN,NaN,NaN
4,file:///home/fmind/mlops-python-package/mlruns...,2024-12-14 14:41:49.910,300971336194126583,active,4753bf6b106845d19eca79b7cb329343,Training,4753bf6b106845d19eca79b7cb329343,2024-12-14 14:41:23.914,FINISHED,fmind,{'training_mean_squared_error': 1.280665585452...,"{'conf_file': 'confs/training.yaml', 'memory':...","{'mlflow.user': 'fmind', 'mlflow.source.name':...",25.996,fmind,file:///home/fmind/mlops-python-package,PROJECT,main,1a4b8a25a32f0e933a7db9ec47f763cc7bd17d1c,git@github.com:fmind/mlops-python-package,git@github.com:fmind/mlops-python-package,Training,virtualenv,local,Pipeline,sklearn.pipeline.Pipeline,"[{""run_id"": ""4753bf6b106845d19eca79b7cb329343""...",NaN,NaN


In [7]:
models = client.search_registered_models(
    max_results=MAX_RESULTS, order_by=["creation_timestamp DESC"]
)
models = [dict(model) for model in models]
models = (
    pd.DataFrame(models)
    .assign(
        creation_timestamp=lambda data: pd.to_datetime(data["creation_timestamp"], unit="ms"),
        last_updated_timestamp=lambda data: pd.to_datetime(
            data["last_updated_timestamp"], unit="ms"
        ),
    )
    .drop(columns=["latest_versions"])
)
print(models.shape)
models

(1, 6)


,aliases,creation_timestamp,description,last_updated_timestamp,name,tags
0,{'Champion': '3'},2024-12-14 14:37:45.482,,2024-12-14 14:41:56.590,bikes,{}


In [8]:
versions = client.search_model_versions(
    max_results=MAX_RESULTS, order_by=["creation_timestamp DESC"]
)
versions = [dict(version) for version in versions]
versions = pd.DataFrame(versions).assign(
    aliases=lambda data: data["aliases"].map(lambda x: x[0] if len(x) else None),
    creation_timestamp=lambda data: pd.to_datetime(data["creation_timestamp"], unit="ms"),
    last_updated_timestamp=lambda data: pd.to_datetime(data["last_updated_timestamp"], unit="ms"),
)
print(versions.shape)
versions.head()

(3, 14)


,aliases,creation_timestamp,current_stage,description,last_updated_timestamp,name,run_id,run_link,source,status,status_message,tags,user_id,version
0,Champion,2024-12-14 14:41:48.936,None,,2024-12-14 14:41:48.936,bikes,4753bf6b106845d19eca79b7cb329343,,file:///home/fmind/mlops-python-package/mlruns...,READY,,{},,3
1,None,2024-12-14 14:39:49.417,None,,2024-12-14 14:39:49.417,bikes,ae9f673f6b264240bb79ffd94ca49fe2,,file:///home/fmind/mlops-python-package/mlruns...,READY,,{},,2
2,None,2024-12-14 14:37:45.487,None,,2024-12-14 14:37:45.487,bikes,d2aed973c12b475f849743599f7bc973,,file:///home/fmind/mlops-python-package/mlruns...,READY,,{},,1


# DASHBOARDS

In [9]:
px.strip(
    experiments,
    x="creation_time",
    color="lifecycle_stage",
    hover_name="name",
    hover_data=experiments.columns,
    title="Experiment Creation Time",
)

In [10]:
px.strip(
    models,
    x="creation_timestamp",
    hover_name="name",
    hover_data=models.columns,
    title="Model Creation Timestamp",
)

In [11]:
px.strip(
    models,
    x="creation_timestamp",
    hover_name="name",
    hover_data=models.columns,
    title="Model Creation Timestamp",
)

In [12]:
px.strip(
    versions,
    x="creation_timestamp",
    color="name",
    hover_name="name",
    hover_data=versions.columns,
    title="Version Creation Timestamp",
)

In [13]:
px.strip(
    runs,
    x="start_time",
    color="experiment_id",
    hover_name="run_name",
    hover_data=runs.columns,
    title="Run Start Time",
)

In [14]:
px.strip(
    runs,
    x="run_time_secs",
    color="run_name",
    hover_name="run_id",
    hover_data=runs.columns,
    title="Run Time",
)

In [15]:
px.bar(runs, x="estimator_class", title="Run Estimator Class Distribution")